<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br>
<b> LSST Crowded Fields: Selecting 47 Tuc Visits </b><br>
Select representative visits from LSSTComCam 47 Tuc data for crowded field processing. <br> <br>

Contact author: Audrey Budlong <br>
Last verified to run: 10 February 2026 <br>

*DP1 Reference: https://dp1.lsst.io/tutorials/notebook/301/notebook-301-1.html*

### Notebook Contents:
1. Imports
2. Setup
3. Define Butler
4. Tap Service
5. 47 Tuc Center
6. Select 47 Tuc Visits
7. Display Coadd
8. Coverage Map
9. Objects
10. KDE Plots
11. Select Visits

### 1. Imports

In [ ]:
import lsst.geom as geom
import lsst.afw.display as afw_display
import lsst.sphgeom as sphgeom
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import numpy as np
import itertools
import seaborn as sns

from lsst.daf.butler import Butler
from lsst.rsp import get_tap_service
from matplotlib.colors import to_rgba
from matplotlib.patches import Polygon
from lsst.utils.plotting import (
    get_multiband_plot_colors,
    get_multiband_plot_symbols,
    get_multiband_plot_linestyles,
)

### 2. Setup

In [ ]:
plt.style.use('seaborn-v0_8-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
plt.rcParams['font.family'] = 'serif'

In [ ]:
filter_colors = get_multiband_plot_colors()
filter_symbols = get_multiband_plot_symbols()
filter_linestyles = get_multiband_plot_linestyles()

In [ ]:
afw_display.setDefaultBackend('firefly')
display = afw_display.Display(frame=1)

### 3. Define Butler

In [ ]:
collections = [
                "LSSTComCam/DP1/defaults",
                "LSSTComCam/runs/DRP/DP1/w_2025_17/DM-50530",
                "skymaps",
            ]

instrument="LSSTComCam"
skymap="lsst_cells_v1"
repo="/repo/main"

butler = Butler(repo, instrument=instrument, collections=collections, skymap=skymap)

### 4. Tap Service

In [ ]:
service = get_tap_service("tap")
assert service is not None

### 5. 47 Tuc Center

In [ ]:
ra_cen = 6.128
dec_cen = -72.090
radius = 1.0

region = sphgeom.Region.from_ivoa_pos(f"CIRCLE {ra_cen} {dec_cen} {radius}")

### 6. Select 47 Tuc Visits

In [ ]:
query = """
        SELECT visit, band, expMidptMJD, expMidpt
        FROM dp1.Visit
        WHERE CONTAINS(POINT('ICRS', ra, dec),CIRCLE('ICRS', {}, {}, {}))=1
        ORDER BY expMidptMJD
        """.format(ra_cen, dec_cen, radius)

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()

In [ ]:
visits = job.fetch_result().to_table()
print(f"Total number of visits: {len(visits)}")

In [ ]:
visits

In [ ]:
all_bands = np.array([visit['band'] for visit in visits], dtype=str)
unique_bands, counts = np.unique(all_bands, return_counts=True)

for band, count in zip(unique_bands, counts):
    print(band, count)

In [ ]:
field_filters = ['g', 'r', 'i', 'y']

In [ ]:
visits_df = visits.to_pandas()
visits_df

In [ ]:
allVisits = visits_df["visit"]
len(allVisits)

In [ ]:
[print(v) for v in allVisits]

### 7. Display Coadd

In [ ]:
query = "patch.region OVERLAPS region AND band='r'"
coadd_datasetrefs = butler.query_datasets("deep_coadd",
                                          where=query,
                                          bind={"region": region},
                                          with_dimension_records=True,
                                          order_by=["patch.tract"])

In [ ]:
len(coadd_datasetrefs)

In [ ]:
coadd = butler.get(coadd_datasetrefs[0])

In [ ]:
display.mtv(coadd)
display.setMaskTransparency(100)

### 8. Coverage Map

In [ ]:
hspmap_rexptime = butler.get('deepCoadd_exposure_time_consolidated_map_sum',
                             skymap='lsst_cells_v1', band='r')
hspmap_rmaglim = butler.get('deepCoadd_psf_maglim_consolidated_map_weighted_mean',
                            skymap='lsst_cells_v1', band='r')

In [ ]:
dec_size = radius
ra_size = dec_size / np.cos(np.radians(dec_cen))
ra_min, ra_max = ra_cen - ra_size, ra_cen + ra_size
dec_min, dec_max = dec_cen - dec_size, dec_cen + dec_size

ra = np.linspace(ra_min, ra_max, 250)
dec = np.linspace(dec_min, dec_max, 250)
x, y = np.meshgrid(ra, dec)

values_rmaglim = hspmap_rmaglim.get_values_pos(x, y)
values_rmaglim = np.where(values_rmaglim < 0.0, np.nan, values_rmaglim)

values_rexptime = hspmap_rexptime.get_values_pos(x, y)
values_rexptime = np.where(values_rexptime < 0.0, np.nan, values_rexptime)

In [ ]:
print('r-band magnitude limit mean/median: %5.2f %5.2f' %
      (np.nanmean(values_rmaglim), np.nanmedian(values_rmaglim)))
print('r-band magnitude min/max: %5.2f %5.2f' %
      (np.nanmin(values_rmaglim), np.nanmax(values_rmaglim)))
print('r-band exposure time (s) mean/median: %5.1f %5.1f' %
      (np.nanmean(values_rexptime), np.nanmedian(values_rexptime)))
print('r-band exposure time min/max (s): %5.1f %5.1f' %
      (np.nanmin(values_rexptime), np.nanmax(values_rexptime)))

In [ ]:
alltracts = [rec.dataId['tract'] for rec in coadd_datasetrefs]
unique_tracts = np.unique(alltracts)

tract_colors = plt.cm.tab10(np.linspace(0, 1, len(unique_tracts)))
linestyles = ['-', '--', '-.', ':'] * ((len(unique_tracts) // 4) + 1)

style_dict = {
    tract: {'color': tract_colors[i], 'linestyle': linestyles[i]}
    for i, tract in enumerate(unique_tracts)
}

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

mesh = ax.pcolormesh(x, y, values_rmaglim, cmap='Greys_r', shading='auto')
fig.colorbar(mesh, ax=ax, label="r-band limiting magnitude (mag)")

for rec in coadd_datasetrefs:
    vertices = rec.dataId.patch.region.getVertices()
    vertices_deg = []
    for vertex in vertices:
        vertices_deg.append([geom.SpherePoint(vertex).getRa().asDegrees(),
                             geom.SpherePoint(vertex).getDec().asDegrees()])
    polygon = Polygon(vertices_deg, closed=True, facecolor='None',
                      edgecolor=style_dict[rec.dataId['tract']]['color'],
                      linestyle=style_dict[rec.dataId['tract']]['linestyle'],
                      linewidth=2)
    ax.add_patch(polygon)

ax.set_xlim(ra_max, ra_min)
ax.set_ylim(dec_min, dec_max)
ax.set_xlabel('RA (deg)')
ax.set_ylabel('Dec (deg)')

handles = [
    mlines.Line2D(
        [], [],
        color=style['color'],
        linestyle=style['linestyle'],
        label=f"Tract {tract}"
    )
    for tract, style in style_dict.items()
]
ax.legend(handles=handles, loc='upper left', ncol=2)

ax.minorticks_on()

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

# mesh = ax.pcolormesh(x, y, values_rmaglim, cmap='Greys_r', shading='auto')
mesh = ax.pcolormesh(x, y, values_rmaglim, cmap='viridis', shading='auto')
fig.colorbar(mesh, ax=ax, label="r-band limiting magnitude (mag)")

for rec in coadd_datasetrefs:
    vertices = rec.dataId.patch.region.getVertices()
    vertices_deg = []
    for vertex in vertices:
        vertices_deg.append([geom.SpherePoint(vertex).getRa().asDegrees(),
                             geom.SpherePoint(vertex).getDec().asDegrees()])
    polygon = Polygon(vertices_deg, closed=True, facecolor='None',
                      edgecolor=style_dict[rec.dataId['tract']]['color'],
                      linestyle=style_dict[rec.dataId['tract']]['linestyle'],
                      linewidth=2)
    ax.add_patch(polygon)

    # --- Add patch label ---
    # Compute centroid (average of RA, Dec of vertices)
    ra_vals, dec_vals = zip(*vertices_deg)
    ra_center = np.mean(ra_vals)
    dec_center = np.mean(dec_vals)
    patch_label = rec.dataId['patch']  # assumes patch ID is accessible like this

    ax.text(
        ra_center, dec_center, str(patch_label),
        color=style_dict[rec.dataId['tract']]['color'],
        fontsize=8,
        ha='center', va='center', alpha=0.7
    )

ax.set_xlim(ra_max, ra_min)
ax.set_ylim(dec_min, dec_max)
ax.set_xlabel('RA (deg)')
ax.set_ylabel('Dec (deg)')

handles = [
    mlines.Line2D(
        [], [],
        color=style['color'],
        linestyle=style['linestyle'],
        label=f"Tract {tract}"
    )
    for tract, style in style_dict.items()
]
ax.legend(handles=handles, loc='upper left', ncol=2)

ax.minorticks_on()

plt.tight_layout()
plt.show()

### 9. Objects

In [ ]:
query = """SELECT objectId, coord_ra, coord_dec, ebv,
        {}_psfMag, {}_cModelMag, {}_psfMag, {}_cModelMag,
        {}_psfMag, {}_cModelMag, {}_psfMag, {}_cModelMag,
        refExtendedness
        FROM dp1.Object
        WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec),
              CIRCLE('ICRS', {}, {}, {})) = 1
        ORDER BY objectId ASC
        """.format(field_filters[0], field_filters[0],
                   field_filters[1], field_filters[1],
                   field_filters[2], field_filters[2],
                   field_filters[3], field_filters[3],
                   ra_cen, dec_cen, radius)

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()

In [ ]:
objtab = job.fetch_result().to_table()

In [ ]:
obj_df = objtab.to_pandas()
len(obj_df)

In [ ]:
obj_df.head()

### 10. KDE Plots

In [ ]:
sns.kdeplot(data=obj_df, x='coord_ra', y='coord_dec')

In [ ]:
sns.kdeplot(data=obj_df, x='coord_ra', y='coord_dec', fill=True, cmap="mako", cbar=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

# --- KDE background ---
sns.kdeplot(
    data=obj_df,
    x='coord_ra',
    y='coord_dec',
    fill=True,
    cmap='viridis',
    cbar=True,
    ax=ax
)

# --- Overplot coadd polygons ---
for rec in coadd_datasetrefs:
    vertices = rec.dataId.patch.region.getVertices()
    vertices_deg = []
    for vertex in vertices:
        vertices_deg.append([
            geom.SpherePoint(vertex).getRa().asDegrees(),
            geom.SpherePoint(vertex).getDec().asDegrees()
        ])
    
    # Example: use tract color scheme
    color = style_dict[rec.dataId['tract']]['color']
    polygon = Polygon(
        vertices_deg,
        closed=True,
        facecolor='none',                     # transparent fill
        edgecolor=to_rgba(color, 0.8),        # colored edge
        linestyle=style_dict[rec.dataId['tract']]['linestyle'],
        linewidth=1.8
    )
    ax.add_patch(polygon)

    # Optional: add patch number
    ra_vals, dec_vals = zip(*vertices_deg)
    ra_center, dec_center = np.mean(ra_vals), np.mean(dec_vals)
    patch_label = rec.dataId['patch']

    ax.text(
        ra_center, dec_center, str(patch_label),
        color=to_rgba(color, 0.9),
        fontsize=8,
        ha='center', va='center'
    )

# --- Create legend handles for each tract ---
handles = [
    plt.Line2D(
        [], [], 
        color=style['color'], 
        linestyle=style['linestyle'],
        linewidth=2,
        label=f"Tract {tract}"
    )
    for tract, style in style_dict.items()
]

ax.legend(handles=handles, loc='upper left', ncol=2, frameon=True)

# --- Axes setup ---
ax.set_xlim(ra_max, ra_min)
ax.set_ylim(dec_min, dec_max)
ax.set_xlabel('RA (deg)')
ax.set_ylabel('Dec (deg)')
ax.minorticks_on()

plt.tight_layout()
plt.show()


### 11. Select Visits

In [ ]:
tract = 453
patches = [63, 64, 65]

In [ ]:
query_47tuc_p63 = "patch.region OVERLAPS region AND patch=63 AND tract=453 AND band='r'"
query_47tuc_p64 = "patch.region OVERLAPS region AND patch=64 AND tract=453 AND band='r'"
query_47tuc_p65 = "patch.region OVERLAPS region AND patch=65 AND tract=453 AND band='r'"
coadd_datasetrefs_p63 = butler.query_datasets("deep_coadd",
                                              where=query_47tuc_p63,
                                              bind={"region": region},
                                              with_dimension_records=True,
                                              order_by=["patch.tract"])
coadd_datasetrefs_p64 = butler.query_datasets("deep_coadd",
                                              where=query_47tuc_p64,
                                              bind={"region": region},
                                              with_dimension_records=True,
                                              order_by=["patch.tract"])
coadd_datasetrefs_p65 = butler.query_datasets("deep_coadd",
                                              where=query_47tuc_p65,
                                              bind={"region": region},
                                              with_dimension_records=True,
                                              order_by=["patch.tract"])

In [ ]:
p63_visits = butler.get("deep_coadd.coaddInputs", dataId=coadd_datasetrefs_p63[0].dataId).ccds["visit"]
p64_visits = butler.get("deep_coadd.coaddInputs", dataId=coadd_datasetrefs_p64[0].dataId).ccds["visit"]
p65_visits = butler.get("deep_coadd.coaddInputs", dataId=coadd_datasetrefs_p65[0].dataId).ccds["visit"]

In [ ]:
query_47tuc_p46 = "patch.region OVERLAPS region AND patch=46 AND tract=453 AND band='r'"

coadd_datasetrefs_p46 = butler.query_datasets("deep_coadd",
                                              where=query_47tuc_p46,
                                              bind={"region": region},
                                              with_dimension_records=True,
                                              order_by=["patch.tract"])

p46_visits = butler.get("deep_coadd.coaddInputs", dataId=coadd_datasetrefs_p46[0].dataId).ccds["visit"]
p46_visits